# Download and Process the IMDB Dataset

In [15]:
import os
import re
import string
import random
from collections import defaultdict, Counter
import math
from math import log, exp

n = 3


In [16]:

def load_imdb_unsup_sentences(folder_path):
    """
    Loads text files from the IMDB 'unsup' (unsupervised) folder.
    split text by newline, strips text, and returns a list of raw lines.
    replace <br /> tags with special token <nl> token.
    """
    all_sentences = []
    for file in os.listdir(folder_path):
        text = open(os.path.join(folder_path,file),"r",encoding="utf-8")
        for line in text.readlines():
            all_sentences.append(line.replace("<br />"," <nl> ").strip())
         
    return all_sentences

def remove_punctuation(text):
    """
    Removes punctuation from the text,
    but keeps <nl> tokens intact.
    """
    text = text.replace(" <nl> ", " xnl ")  # Temporarily separate <nl> to preserve it
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.replace(" xnl ", " <nl> ")  # Restore <nl> in its place
    return text

def build_vocabulary(sentences):
    """
    lower each sentence,
    Splits each sentence on whitespace, removes punctuation,
    and builds a set of unique tokens (vocabulary).
    """
    vocab = set()
    sentence:str
    for sentence in sentences:
        sentence = remove_punctuation(sentence).lower()
        for word in sentence.split():
            if word != "<nl>":
                # print("fdssfsdfsdfsdfsdf")
                vocab.add(word)
        
    return vocab

def tokinize(sentences, vocab, unknown="<UNK>"):
    """
    lower each sentence,
    Splits each sentence on whitespace, removes punctuation,
    and replaces tokens not in the vocabulary with unknowen token.
    Returns the list of tokenized sentences.
    """
    tokenized_sentences = []
    sentence:str
    for sentence in sentences:
        sentence = remove_punctuation(sentence).lower()
        for word in sentence.split():
            if word not in vocab and word != "<nl>":
                sentence.replace(word,unknown)
        tokenized_sentences.append(sentence.split())

    return tokenized_sentences

In [17]:
#imdb_folder = "imdb_data/train/unsup"
imdb_folder = "S:/imdb_data/train/unsup"

sentences = load_imdb_unsup_sentences(imdb_folder)

print(f"Number of raw sentences loaded: {len(sentences)}")
print(f"Example (first 2 sentences):\n{sentences[:1]}")


Number of raw sentences loaded: 50000
Example (first 2 sentences):
['I admit, the great majority of films released before say 1933 are just not for me. Of the dozen or so "major" silents I have viewed, one I loved (The Crowd), and two were very good (The Last Command and City Lights, that latter Chaplin circa 1931). <nl>  <nl> So I was apprehensive about this one, and humor is often difficult to appreciate (uh, enjoy) decades later. I did like the lead actors, but thought little of the film. <nl>  <nl> One intriguing sequence. Early on, the guys are supposed to get "de-loused" and for about three minutes, fully dressed, do some schtick. In the background, perhaps three dozen men pass by, all naked, white and black (WWI ?), and for most, their butts, part or full backside, are shown. Was this an early variation of beefcake courtesy of Howard Hughes?']


In [18]:
assert len(sentences) == 50000, "Expected 50,000 sentences from the unsup folder."

In [19]:
random.seed(42)

def split_data(sentences, test_split=0.1):
    """
      shuffle the sentences
      split them into train and test sets (first 1-test_split of the data is the training)
      return the train and test sets
    """
    random.shuffle(sentences)
    train_split_length = int(len(sentences)*(1-test_split))
    train_sentences, test_sentences = sentences[:train_split_length],sentences[train_split_length:]
    
    return train_sentences, test_sentences


In [20]:
train_sentences, test_sentences = split_data(sentences)

print(f"Number of training sentences: {len(train_sentences)}")
print(f"Number of test sentences: {len(test_sentences)}")

Number of training sentences: 45000
Number of test sentences: 5000


In [21]:
assert len(train_sentences) == 45000, "Expected 45,000 sentences for training."
assert len(test_sentences) == 5000, "Expected 5,000 sentences for testing."


In [22]:
vocab = build_vocabulary(train_sentences)
tokenized_sentences = tokinize(train_sentences, vocab)

print(f"Vocabulary size: {len(vocab)}")
print(f"Example tokens from first sentence: {tokenized_sentences[0][:90] if tokenized_sentences else 'No tokens loaded'} ...")

# for word in vocab:
#     if word == "nl":
#         print("fdssfsdfsdfsdfsdf")


Vocabulary size: 161292
Example tokens from first sentence: ['having', 'first', 'seen', 'the', 'directors', '12min', 'take', 'on', 'poes', 'fall', 'of', 'the', 'house', 'of', 'usher', 'i', 'was', 'looking', 'forward', 'to', 'seeing', 'this', 'one', 'too', 'and', 'wasnt', 'disappointed', 'at', 'all', 'though', 'perhaps', 'not', 'quite', 'up', 'to', 'the', 'same', 'level', 'of', 'artistic', 'attainment', 'as', 'usher', 'it', 'is', 'nevertheless', 'very', 'much', 'in', 'the', 'same', 'vein', '<nl>', '<nl>', 'like', 'the', 'usher', 'the', 'viewer', 'should', 'be', 'familiar', 'beforehand', 'with', 'the', 'story', 'on', 'which', 'it', 'is', 'based', 'in', '1928', 'the', 'directors', 'watson', 'and', 'webber', 'could', 'have', 'safely', 'assumed', 'the', 'audiences', 'knowledge', 'of', 'the', 'biblical', 'tale', 'interestingly'] ...


In [23]:
assert len(vocab) == 161292, "Expected a vocabulary size of 171,591."
assert len(tokenized_sentences) == 45000, "Expected tokenized sentences count to match raw sentences."

example = "I love Natural language processing, and i want to be a great engineer."
assert len(example) == 70, "Example sentence length (in characters) does not match the expected 70."

example_tokens = tokinize([example], vocab)[0]

assert len(example_tokens) == 13, "Token count for the example sentence does not match the expected 13."


In [38]:

def pad_sentence(tokens, n):
    """
    Pads a list of tokens with <s> at the start (n-1 times)
    and </s> at the end (once).
    For example, if n=3, you add 2 <s> tokens at the start.
    """
    #! Probably it should return a list not a string.
    #! The provided code returns a string
    padded = ["<s>"]*(n-1) + tokens + ["</s>"]
    return padded

def build_ngram_counts(tokenized_sentences, n,vocab):
    """
    Builds n-gram counts and (n-1)-gram counts from the given tokenized sentences.
    Each sentence is padded with <s> and </s>.

    Args:
        tokenized_sentences: list of lists, where each sub-list is a tokenized sentence.
        n: the order of the n-gram (e.g., 2 for bigrams, 3 for trigrams).
        vocab: set of known words. If provided, you can choose to handle out-of-vocab tokens.

    Returns:
        ngram_counts: Counter of n-grams (tuples of length n).
        context_counts: Counter of (n-1)-gram contexts.
    """
    ngram_counts = Counter()
    context_counts = Counter()
    for sentence in tokenized_sentences:
        padded_sentence = pad_sentence(sentence,n)
        
        for i in range(len(padded_sentence) - n + 1):
            ngram = tuple(padded_sentence[i:i + n])  
            context = tuple(padded_sentence[i:i + n - 1]) 
            

            context_counts[context] += 1
            
            if "<nl><nl>" in "".join(ngram):  
                continue
            # if "<nl>" in context:
            #     continue

            ngram_counts[ngram] += 1
            
    
        
    return ngram_counts, context_counts

def laplace_probability(ngram, ngram_counts, context_counts, vocab_size, alpha=1.0):
    """
    Computes the probability of an n-gram using Laplace (add-alpha) smoothing.

    P(w_i | w_{i-(n-1)}, ..., w_{i-1}) =
        (count(ngram) + alpha) / (count(context) + alpha * vocab_size)

    Args:
        ngram: tuple of tokens representing the n-gram
        ngram_counts: Counter of n-grams
        context_counts: Counter of (n-1)-gram contexts
        vocab_size: size of the vocabulary
        alpha: smoothing parameter (1.0 = add-1 smoothing)

    Returns:
        Probability of the given n-gram.
    """
    ngram_count = ngram_counts.get(ngram, 0) 
    context = ngram[:-1]
    context_count = context_counts.get(context, 0) 


    prob = (ngram_count + alpha) / (context_count + (alpha * context_count) + 10e-10)


    return prob




In [25]:

ngram_counts, context_counts = build_ngram_counts(tokenized_sentences, n, vocab=vocab)
context_counts = {context: count for context, count in context_counts.items() if "<nl>" not in context}
print(f"Number of bigrams: {len(ngram_counts)}")
print(f"Number of contexts: {len(context_counts)}")


Number of bigrams: 6090230
Number of contexts: 2249327


In [26]:
print(f"Example bigrams: {list(ngram_counts.items())[:150]}")
print(f"Example contexts: {list(context_counts.items())[:15]}")

Example bigrams: [(('<s>', '<s>', 'having'), 137), (('<s>', 'having', 'first'), 1), (('having', 'first', 'seen'), 3), (('first', 'seen', 'the'), 2), (('seen', 'the', 'directors'), 1), (('the', 'directors', '12min'), 1), (('directors', '12min', 'take'), 1), (('12min', 'take', 'on'), 1), (('take', 'on', 'poes'), 1), (('on', 'poes', 'fall'), 1), (('poes', 'fall', 'of'), 1), (('fall', 'of', 'the'), 26), (('of', 'the', 'house'), 120), (('the', 'house', 'of'), 74), (('house', 'of', 'usher'), 7), (('of', 'usher', 'i'), 1), (('usher', 'i', 'was'), 1), (('i', 'was', 'looking'), 116), (('was', 'looking', 'forward'), 74), (('looking', 'forward', 'to'), 275), (('forward', 'to', 'seeing'), 118), (('to', 'seeing', 'this'), 38), (('seeing', 'this', 'one'), 15), (('this', 'one', 'too'), 24), (('one', 'too', 'and'), 4), (('too', 'and', 'wasnt'), 1), (('and', 'wasnt', 'disappointed'), 5), (('wasnt', 'disappointed', 'at'), 2), (('disappointed', 'at', 'all'), 5), (('at', 'all', 'though'), 9), (('all', 'th

In [27]:
from math import log, exp

def predict_next_token(
    context_tokens,
    ngram_counts,
    context_counts,
    vocab,
    n,
    alpha=1.0,
    top_k=5
):
    """
    Given a list of context tokens, predict the next token using the n-gram model.
    Returns the top_k predictions as (token, probability).
    """

    candidates = []
    context = tuple(context_tokens[-(n - 1):])  # Extract the last (n-1) tokens as context
    
    vocab_size = len(vocab)
    
    for token in vocab:
        ngram = context + (token,)  # Create an n-gram candidate
        prob = laplace_probability(ngram, ngram_counts, context_counts, vocab_size, alpha)
        candidates.append((token, prob))
    
    # Sort by probability in descending order and return top_k results
    candidates.sort(key=lambda x: x[1], reverse=True)
    return candidates[:top_k]


def generate_text_with_limit(
    start_tokens,
    ngram_counts,
    context_counts,
    vocab,
    n,
    alpha=1.0,
    max_length=20
):
    """
    Generates text from an n-gram model until it sees </s>
    or reaches a maximum total length (max_length).

    Args:
      start_tokens (list): initial context to begin generation
      ngram_counts (Counter): trained n-gram counts
      context_counts (Counter): trained (n-1)-gram counts
      vocab (set): the model vocabulary
      n (int): n-gram order, 2 for bigram, 3 for trigram, etc.
      alpha (float): Laplace smoothing parameter
      max_length (int): maximum number of tokens to generate (including start_tokens)

    Returns:
      A list of tokens representing the generated sequence.
    """
    generated = list(start_tokens)  # Initialize with given start tokens
    
    while len(generated) < max_length:
        predictions = predict_next_token(generated, ngram_counts, context_counts, vocab, n, alpha)
        
        next_token = predictions[0][0]  # Select the most probable token
        
        if next_token == "</s>": 
            break
        
        generated.append(next_token)  
    
    return generated

context = ["i", "love"]
generated_seq = generate_text_with_limit(
    start_tokens=context,
    ngram_counts=ngram_counts,
    context_counts=context_counts,
    vocab=vocab,
    n=n,
    alpha=1.0,
    max_length=128
)

print("Generated Sequence:", generated_seq)


Generated Sequence: ['i', 'love', 'the', 'way', 'the', 'film', 'is', 'a', 'very', 'good', 'and', 'the', 'film', 'is', 'a', 'very', 'good', 'and', 'the', 'film', 'is', 'a', 'very', 'good', 'and', 'the', 'film', 'is', 'a', 'very', 'good', 'and', 'the', 'film', 'is', 'a', 'very', 'good', 'and', 'the', 'film', 'is', 'a', 'very', 'good', 'and', 'the', 'film', 'is', 'a', 'very', 'good', 'and', 'the', 'film', 'is', 'a', 'very', 'good', 'and', 'the', 'film', 'is', 'a', 'very', 'good', 'and', 'the', 'film', 'is', 'a', 'very', 'good', 'and', 'the', 'film', 'is', 'a', 'very', 'good', 'and', 'the', 'film', 'is', 'a', 'very', 'good', 'and', 'the', 'film', 'is', 'a', 'very', 'good', 'and', 'the', 'film', 'is', 'a', 'very', 'good', 'and', 'the', 'film', 'is', 'a', 'very', 'good', 'and', 'the', 'film', 'is', 'a', 'very', 'good', 'and', 'the', 'film', 'is', 'a', 'very', 'good', 'and', 'the', 'film', 'is', 'a', 'very']


In [39]:
def calculate_perplexity(
    tokenized_sentences,
    ngram_counts,
    context_counts,
    vocab_size,
    n,
    alpha=1.0
):
    """
    Calculates the perplexity of an n-gram model (with Laplace smoothing)
    on a list of tokenized sentences.

    Args:
      tokenized_sentences: List of lists of tokens.
      ngram_counts: Counter of n-grams.
      context_counts: Counter of (n-1)-grams.
      vocab_size: Size of the vocabulary.
      n: n-gram order.
      alpha: Laplace smoothing parameter.

    Returns:
      A float representing the perplexity on the given dataset.
    """

    log_prob_sum = 0.0
    word_count = 0  

    for sentence in tokenized_sentences:
        padded_sentence = pad_sentence(sentence, n)

        for i in range(len(padded_sentence) - n + 1):
            ngram = tuple(padded_sentence[i:i + n])
            prob = laplace_probability(ngram, ngram_counts, context_counts, vocab_size, alpha)

            log_prob_sum += log(prob)  
            word_count += 1

    perplexity = exp((-1/word_count)*log_prob_sum ) if word_count > 0 else float('inf')
    return perplexity

# **Analysis**
use different n and rerun the code and write down your analysis

In [40]:
n = range(1,4) # N from 1 to 3
tokenized_test_sentences = tokinize(test_sentences, vocab)
for i in n:
    ngram_counts, context_counts = build_ngram_counts(tokenized_sentences, i, vocab=vocab)
    padded_test_sentences = [pad_sentence(sentence, i) for sentence in tokenized_sentences]
    pp= calculate_perplexity(tokenized_test_sentences, ngram_counts, context_counts, len(vocab), i, alpha=1.0)
    print(f"Perplexity for n={i}: {pp}")


Perplexity for n=1: 2416.2647330664254
Perplexity for n=2: 244.05077090940617
Perplexity for n=3: 1.0592658345689983


## Using NLTK Library to evaluate the preplexity 

In [ ]:
import nltk
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import Laplace

In [ ]:
for i in n:
    ngram_counts, context_counts = build_ngram_counts(tokenized_sentences, i, vocab=vocab)

    nltk_train_data, nltk_vocab = padded_everygram_pipeline(i, tokenized_sentences)
    nltk_model = Laplace(i) 
    nltk_model.fit(nltk_train_data, nltk_vocab)
    
    nltk_test_data, _ = padded_everygram_pipeline(i, tokenized_test_sentences)
    nltk_perplexity = nltk_model.perplexity(next(nltk_test_data))
    print(f"NLTK Model Perplexity with n={i}: {nltk_perplexity}")
    

NLTK Model Perplexity with n=1 : 958.6964613574661
NLTK Model Perplexity with n=2 : 1661.488686634242
NLTK Model Perplexity with n=3 : 4194.543335552027


### Notes on the Results

Why Perplexity Increases with n:

- Data Sparsity:
As n increases, the number of possible n-grams grows exponentially.
Many n-grams in the test data may not have been seen in the training data, leading to more cases where probabilities are extremely low or zero (even with smoothing).
- Context Specificity:
Larger n means the model relies on longer context windows.
If an (n-1)-gram context was rare in training, then the corresponding n-gram probability may be poorly estimated, leading to a higher perplexity.
- Laplace Smoothing Effect:
Smoothing helps handle unseen n-grams, but when n increases, the denominator in Laplace smoothing (context_counts[context] + alpha * vocab_size) also increases, making probabilities even smaller.

### NLTK Perplexity Calculation Method

The NLTK uses 2**(cross_entropy for the text). This results in a different ranges, but still the perplexity increases as N increases. 